<a href="https://colab.research.google.com/github/MariaIsabel-90/Curso-analisedados/blob/main/Aula_spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Primeiro passo: instalar o Java 8

Segundo passo: instalar o Apache spark com Hadoop

Configurar as variáveis de ambiente

In [1]:
!apt-get update -qq
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"



W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


## Conectar a um cluster do pyspark

In [2]:
import findspark
findspark.init()


In [ ]:
from pyspark import SparkContext
spark_contexto = SparkContext()


NameError: name 'spark_context' is not defined

In [ ]:
print(spark_contexto)
print(spark_contexto.version)

<SparkContext master=local[*] appName=pyspark-shell>
3.1.2


In [ ]:
#criar uma spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
print(spark)

In [ ]:
dataset = spark.read.csv('/content/sample_data/california_housing_test.csv',inferSchema=True, header =True)

In [ ]:
#ler cabeçalho
dataset.head()

Row(longitude=-122.05, latitude=37.37, housing_median_age=27.0, total_rooms=3885.0, total_bedrooms=661.0, population=1537.0, households=606.0, median_income=6.6085, median_house_value=344700.0)

In [ ]:
#contar a quantidade de linhas
dataset.count()

3000

In [ ]:
#criar tabela temporária com os dados
dataset.createOrReplaceTempView('tabela_temporaria')
print(spark.catalog.listTables())

[Table(name='tabela_temporaria', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]


In [ ]:
#fazer uma query na tabela temporária
query="FROM tabela_temporaria SELECT latitude,longitude LIMIT 3"
saida = spark.sql(query)
saida.show()


+--------+---------+
|latitude|longitude|
+--------+---------+
|   37.37|  -122.05|
|   34.26|   -118.3|
|   33.78|  -117.81|
+--------+---------+



### Queremos encontrar as casas com a maior quantidade de quartos

In [ ]:
query1 = "SELECT MAX(total_rooms) as maximo_quartos FROM tabela_temporaria"
q_max_quartos = spark.sql(query1)
pd_max_quartos = q_max_quartos.toPandas()
print('A quantidade máxima de quartos é:{}'.format(pd_max_quartos['maximo_quartos']))
qtd_max_quartos  = int(pd_max_quartos.loc[0,'maximo_quartos'])

A quantidade máxima de quartos é:0    30450.0
Name: maximo_quartos, dtype: float64


In [ ]:
#procurar a localização dessas casas com maior numero de quartos
query2 = "Select longitude, latitude from tabela_temporaria WHERE total_rooms= "+str(qtd_max_quartos)
local_maximo_quartos = spark.sql(query2)
pd_local_maximo_quartos = local_maximo_quartos.toPandas()
print(pd_local_maximo_quartos.head())

   longitude  latitude
0     -117.2     33.58


In [ ]:
import pandas as pd
import numpy as np
pd.DataFrame.iteritems = pd.DataFrame.items

In [ ]:
media = 0
desvpad = 0.1
pd_temporario = pd.DataFrame(np.random.normal(media,desvpad,100))
spark_temporario = spark.createDataFrame(pd_temporario)
print(spark.catalog.listTables())
spark_temporario.createOrReplaceTempView('nova_tabela_temporaria')
print(spark.catalog.listTables())

[Table(name='tabela_temporaria', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]
[Table(name='nova_tabela_temporaria', database=None, description=None, tableType='TEMPORARY', isTemporary=True), Table(name='tabela_temporaria', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]


In [ ]:
spark.stop()

### Aula de mapreduce

In [4]:
from pyspark import SparkContext
spark_contexto = SparkContext()

In [5]:
import numpy as np

In [6]:
#criar vetor numerico
vetor = np.array([10,20,30,40,50])

In [7]:
#criar um RDD
paralelo = spark_contexto.parallelize(vetor)

In [8]:
print(paralelo)

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274


In [9]:
mapa = paralelo.map(lambda x: x**2 + x)

In [10]:
#verificar os dados
mapa.collect()

[110, 420, 930, 1640, 2550]

#### Exemplo 2: listas

In [11]:
paralelo = spark_contexto.parallelize(['distribuida', 'distribuida', 'spark', 'rdd', 'spark','spark'])

In [12]:
#associar o numero 1 a cada palavra
funcao_lambda = lambda x: (x,1)

In [13]:
from operator import add

In [14]:
mapa2 = paralelo.map(funcao_lambda).reduceByKey(add).collect()

In [16]:
#visualizar os resultados
for (w,c) in mapa2:
  print('{}: {}'.format(w,c))

distribuida: 2
spark: 3
rdd: 1


In [17]:
spark_contexto.stop()

### Aula 4 - Transformações com spark

In [3]:
from pyspark import SparkContext
spark_contexto = SparkContext()

In [4]:
#criar uma lista e criar o RDD
lista = [1,2,3,4,5]
lista_rdd = spark_contexto.parallelize(lista)

In [5]:
#contar os elementos do RDD
lista_rdd.count()

5

In [6]:
par_ordenado = lambda numero:(numero,numero*10)

In [7]:
lista_rdd.flatMap(par_ordenado).collect()

[1, 10, 2, 20, 3, 30, 4, 40, 5, 50]

In [9]:
#aplicar a função par_ordenado ao RDD
lista_rdd.map(par_ordenado).collect()

[(1, 10), (2, 20), (3, 30), (4, 40), (5, 50)]

In [10]:
#fechar a sessão
spark_contexto.stop()